In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import os
import rospy
import rosbag
import cv2
import message_filters
import tf2_ros
import numpy as np # 引入 NumPy 用于点云生成
from cv_bridge import CvBridge
from sensor_msgs.msg import Image, CameraInfo, PointCloud2, PointField # 引入 PointCloud2 和 PointField
import sensor_msgs.point_cloud2 as pc2 # 引入点云工具
from geometry_msgs.msg import TransformStamped

# --- [!!! 用户配置 - 发布节点 !!!] ---
BAG_FILE_PATH = '../2025-12-01-19-01-56.bag'
TARGET_FRAME_INDEX = 50 
FRAME_ID = 'camera_color_frame'
MAP_FRAME_ID = 'map'
TIME_TOLERANCE = rospy.Duration(0.01)

# 发布话题名称
PUB_TOPIC_RGB = '/input/frame_50_rgb'
PUB_TOPIC_DEPTH = '/input/frame_50_depth'
PUB_TOPIC_INFO = '/input/frame_50_info'
PUB_TOPIC_PCD_RAW = '/input/frame_50_raw_rgb_pcd' # 新增：原始点云话题
PUBLISH_RATE_HZ = 10 
DEPTH_SCALE = 1000.0 # 深度转换比例 (mm to m)
# ----------------------------

class BagFramePublisher:
    def __init__(self):
        rospy.init_node('bag_frame_publisher', anonymous=True, log_level=rospy.INFO)
        self.bridge = CvBridge()
        
        # 传感器发布器
        self.pub_rgb = rospy.Publisher(PUB_TOPIC_RGB, Image, queue_size=1)
        self.pub_depth = rospy.Publisher(PUB_TOPIC_DEPTH, Image, queue_size=1)
        self.pub_info = rospy.Publisher(PUB_TOPIC_INFO, CameraInfo, queue_size=1)
        # 点云发布器
        self.pub_pcd_raw = rospy.Publisher(PUB_TOPIC_PCD_RAW, PointCloud2, queue_size=1) 
        self.tf_broadcaster = tf2_ros.TransformBroadcaster()

        # 内存中存储的消息
        self.rgb_msg = None
        self.depth_msg = None
        self.info_msg = None
        self.pc_raw_msg = None # 新增：存储构建好的原始点云消息
        
        # 提取数据到内存
        if self._extract_frame_to_memory():
            self._generate_raw_point_cloud()


    def _extract_frame_to_memory(self):
        # (提取 RGB, Depth, Info 消息到 self.rgb_msg, self.depth_msg, self.info_msg 的逻辑保持不变)
        rospy.loginfo(f"开始离线提取。目标帧索引: {TARGET_FRAME_INDEX}")
        frame_counter = 0
        
        bag_topics = ['/camera/color/image_raw', '/camera/aligned_depth_to_color/image_raw', '/camera/color/camera_info']
        time_buffer = {} 

        try:
            bag = rosbag.Bag(BAG_FILE_PATH, 'r')
            for topic, msg, t in bag.read_messages(topics=bag_topics):
                time_buffer[topic] = (msg, t)
                
                if all(t in time_buffer for t in bag_topics):
                    t_rgb, t_depth, t_info = time_buffer[bag_topics[0]][1], time_buffer[bag_topics[1]][1], time_buffer[bag_topics[2]][1]
                    
                    if abs(t_rgb - t_depth) < TIME_TOLERANCE and abs(t_rgb - t_info) < TIME_TOLERANCE:
                        frame_counter += 1
                        
                        if frame_counter == TARGET_FRAME_INDEX:
                            rospy.loginfo(f"--- 找到目标帧 {TARGET_FRAME_INDEX}，时间: {t_rgb.to_sec()} ---")
                            
                            self.rgb_msg = time_buffer[bag_topics[0]][0]
                            self.depth_msg = time_buffer[bag_topics[1]][0]
                            self.info_msg = time_buffer[bag_topics[2]][0]
                            
                            self.rgb_msg.header.frame_id = FRAME_ID
                            self.depth_msg.header.frame_id = FRAME_ID
                            self.info_msg.header.frame_id = FRAME_ID
                            
                            rospy.loginfo(f"目标帧消息已加载到内存，Frame ID: {FRAME_ID}")
                            return True
                        
                        time_buffer = {} 
        except Exception as e:
            rospy.logerr(f"打开或读取 Bag 文件失败: {e}")
        finally:
            bag.close()
            
        rospy.logwarn(f"目标索引 {TARGET_FRAME_INDEX} 未找到。")
        return False
    
    def _encode_rgb_to_uint32(self, r: int, g: int, b: int) -> int:
        """将 R, G, B 整数分量编码为 UINT32 整数 (BGR 序，兼容 PCL/ROS)。"""
        # 注意：cv_bridge 转换的 cv_rgb 是 BGR 序，但 PCL/ROS 的 RGB 字段通常要求 (R, G, B) 编码为 UINT32
        # 我们使用 (B << 16) | (G << 8) | R 的方式编码，以匹配 ROS 的 PointCloud2 RGB 字段存储方式。
        return (int(b) << 16) | (int(g) << 8) | int(r)
        
    def _generate_raw_point_cloud(self):
        """将内存中的 RGB, 深度, 内参转换为 PointCloud2 消息并存储。"""
        if self.rgb_msg is None or self.depth_msg is None or self.info_msg is None:
            rospy.logerr("缺少传感器消息或内参，无法生成点云。")
            return

        rospy.loginfo("开始生成原始 RGB-D 点云...")

        try:
            # 转换 ROS 消息到 OpenCV/NumPy
            cv_rgb = self.bridge.imgmsg_to_cv2(self.rgb_msg, "bgr8")
            cv_depth = self.bridge.imgmsg_to_cv2(self.depth_msg, "16UC1")
            
            K_matrix = self.info_msg.K
            # 提取 K=[fx, fy, cx, cy]
            fx, fy, cx, cy = K_matrix[0], K_matrix[4], K_matrix[2], K_matrix[5]
            H, W = cv_depth.shape

            points = []
            
            for v in range(H):
                for u in range(W):
                    depth_mm = cv_depth[v, u]
                    if depth_mm == 0: continue
                    
                    Z = float(depth_mm) / DEPTH_SCALE
                    X = (u - cx) * Z / fx
                    Y = (v - cy) * Z / fy
                    
                    # 原始 BGR 颜色
                    b_orig, g_orig, r_orig = cv_rgb[v, u, :].astype(int)
                    # 编码为 UINT32 (ROS BGR 序)
                    rgb_int = self._encode_rgb_to_uint32(r_orig, g_orig, b_orig)

                    points.append((X, Y, Z, rgb_int))
            
            # 定义 PointCloud2 字段
            pcd_fields = [
                PointField('x', 0, PointField.FLOAT32, 1),
                PointField('y', 4, PointField.FLOAT32, 1),
                PointField('z', 8, PointField.FLOAT32, 1),
                PointField('rgb', 12, PointField.UINT32, 1)
            ]
            
            header = rospy.Header()
            header.frame_id = FRAME_ID 
            
            # 创建 PointCloud2 消息并存储
            self.pc_raw_msg = pc2.create_cloud(header, pcd_fields, points)
            rospy.loginfo(f"原始点云消息构建完成，点数: {len(points)}。")

        except Exception as e:
            rospy.logerr(f"生成点云失败: {e}")

    def _publish_tf(self, current_time):
        t = TransformStamped()
        t.header.stamp = current_time
        t.header.frame_id = MAP_FRAME_ID
        t.child_frame_id = FRAME_ID
        t.transform.translation.x = 0.0
        t.transform.translation.y = 0.0
        t.transform.translation.z = 0.0
        t.transform.rotation.x = 0.0
        t.transform.rotation.y = 0.0
        t.transform.rotation.z = 0.0
        t.transform.rotation.w = 1.0 
        self.tf_broadcaster.sendTransform(t)

    def run_publish_loop(self):
        if self.rgb_msg is None or self.pc_raw_msg is None:
            rospy.logerr("缺少目标帧数据或点云，无法发布。")
            return

        rate = rospy.Rate(PUBLISH_RATE_HZ)
        rospy.loginfo(f"开始以 {PUBLISH_RATE_HZ} Hz 持续发布帧 {TARGET_FRAME_INDEX} 的数据，包括原始点云。")

        while not rospy.is_shutdown():
            current_time = rospy.Time.now()
            
            # 1. 更新所有 Header 的时间戳
            self.rgb_msg.header.stamp = current_time
            self.depth_msg.header.stamp = current_time
            self.info_msg.header.stamp = current_time
            self.pc_raw_msg.header.stamp = current_time
            
            # 2. 发布 TF 变换
            self._publish_tf(current_time)
            
            # 3. 发布传感器图像和内参
            self.pub_rgb.publish(self.rgb_msg)
            self.pub_depth.publish(self.depth_msg)
            self.pub_info.publish(self.info_msg)
            
            # 4. 发布原始点云 (新增部分)
            self.pub_pcd_raw.publish(self.pc_raw_msg)
            
            rate.sleep()

if __name__ == '__main__':
    try:
        publisher = BagFramePublisher()
        publisher.run_publish_loop()
    except rospy.ROSInterruptException:
        rospy.loginfo("ROS 中断，Bag 发布节点退出。")

[INFO] [1764752090.271453]: 开始离线提取。目标帧索引: 50
[INFO] [1764752090.482458]: --- 找到目标帧 50，时间: 1764586918.9352887 ---
[INFO] [1764752090.483421]: 目标帧消息已加载到内存，Frame ID: camera_color_frame
[INFO] [1764752090.483915]: 开始生成原始 RGB-D 点云...
[INFO] [1764752091.527826]: 原始点云消息构建完成，点数: 297240。
[INFO] [1764752091.546679]: 开始以 10 Hz 持续发布帧 50 的数据，包括原始点云。


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import os
import rospy
import rosbag
import cv2
import message_filters
import tf2_ros
import numpy as np 
from cv_bridge import CvBridge
from sensor_msgs.msg import Image, CameraInfo, PointCloud2, PointField 
import sensor_msgs.point_cloud2 as pc2 
from geometry_msgs.msg import TransformStamped

# --- [!!! 用户配置 - 发布节点 !!!] ---
BAG_FILE_PATH = '../2025-12-01-19-01-56.bag'
TARGET_FRAME_INDEX_START = 50 # 初始帧索引
FRAME_ID = 'camera_color_frame'
MAP_FRAME_ID = 'map'
TIME_TOLERANCE = rospy.Duration(0.01)

# 动态控制参数
FRAME_INDEX_INCREMENT = 50  # 每次跳跃的帧数
CYCLE_RATE_SEC = 10         # 每 10 秒切换一次帧
PUBLISH_RATE_HZ = 10        # 持续发布的频率
PUB_TOPIC_RGB = '/input/frame_50_rgb'
PUB_TOPIC_DEPTH = '/input/frame_50_depth'
PUB_TOPIC_INFO = '/input/frame_50_info'
PUB_TOPIC_PCD_RAW = '/input/frame_50_raw_rgb_pcd' 
DEPTH_SCALE = 1000.0 
# ----------------------------

class BagFramePublisher:
    def __init__(self):
        rospy.init_node('bag_frame_publisher', anonymous=True, log_level=rospy.INFO)
        self.bridge = CvBridge()
        
        # 帧控制变量
        self.sync_frames_list = [] # 存储所有提取的同步消息列表
        self.max_index = 0
        self.current_index = TARGET_FRAME_INDEX_START

        # 内存中存储的消息 (当前发布的帧数据)
        self.rgb_msg = None
        self.depth_msg = None
        self.info_msg = None
        self.pc_raw_msg = None 
        
        # 传感器发布器
        self.pub_rgb = rospy.Publisher(PUB_TOPIC_RGB, Image, queue_size=1)
        self.pub_depth = rospy.Publisher(PUB_TOPIC_DEPTH, Image, queue_size=1)
        self.pub_info = rospy.Publisher(PUB_TOPIC_INFO, CameraInfo, queue_size=1)
        self.pub_pcd_raw = rospy.Publisher(PUB_TOPIC_PCD_RAW, PointCloud2, queue_size=1) 
        self.tf_broadcaster = tf2_ros.TransformBroadcaster()

        # 1. 提取所有同步数据到内存
        if not self._load_all_sync_frames():
            rospy.signal_shutdown("无法加载数据，节点退出。")
            return
        
        # 2. 初始化第一帧数据
        self._update_current_frame_data()

        # 3. 启动定时器，每 CYCLE_RATE_SEC 秒更新一次帧索引
        rospy.Timer(rospy.Duration(CYCLE_RATE_SEC), self._cycle_timer_callback)
        rospy.loginfo(f"数据循环定时器启动，每 {CYCLE_RATE_SEC} 秒切换下一帧。")


    def _load_all_sync_frames(self):
        """从 Bag 文件中提取所有同步消息，并存储到列表中。"""
        rospy.loginfo(f"开始离线提取所有同步帧...")
        bag_topics = ['/camera/color/image_raw', '/camera/aligned_depth_to_color/image_raw', '/camera/color/camera_info']
        time_buffer = {} 

        try:
            bag = rosbag.Bag(BAG_FILE_PATH, 'r')
            for topic, msg, t in bag.read_messages(topics=bag_topics):
                time_buffer[topic] = (msg, t)
                
                if all(t in time_buffer for t in bag_topics):
                    t_rgb, t_depth, t_info = time_buffer[bag_topics[0]][1], time_buffer[bag_topics[1]][1], time_buffer[bag_topics[2]][1]
                    
                    if abs(t_rgb - t_depth) < TIME_TOLERANCE and abs(t_rgb - t_info) < TIME_TOLERANCE:
                        
                        # 存储 (RGB Msg, Depth Msg, Info Msg)
                        frame_data = (
                            time_buffer[bag_topics[0]][0], 
                            time_buffer[bag_topics[1]][0], 
                            time_buffer[bag_topics[2]][0]
                        )
                        self.sync_frames_list.append(frame_data)
                        
                        time_buffer = {} 
                        
        except Exception as e:
            rospy.logerr(f"打开或读取 Bag 文件失败: {e}")
            return False
        finally:
            bag.close()
            
        self.max_index = len(self.sync_frames_list)
        if self.max_index == 0:
            rospy.logwarn("未找到任何同步帧。")
            return False
            
        rospy.loginfo(f"成功加载 {self.max_index} 帧同步数据。")
        # 确保起始索引在有效范围内
        self.current_index = self.current_index % self.max_index
        return True
    
    def _encode_rgb_to_uint32(self, r: int, g: int, b: int) -> int:
        """Encodes R, G, B components into a UINT32 integer (BGR order for ROS/PCL)."""
        return (int(b) << 16) | (int(g) << 8) | int(r)
        
    def _generate_raw_point_cloud(self, rgb_msg, depth_msg, info_msg):
        """Converts RGB, Depth, and Info messages into a PointCloud2 message."""
        try:
            cv_rgb = self.bridge.imgmsg_to_cv2(rgb_msg, "bgr8")
            cv_depth = self.bridge.imgmsg_to_cv2(depth_msg, "16UC1")
            
            K_matrix = info_msg.K
            fx, fy, cx, cy = K_matrix[0], K_matrix[4], K_matrix[2], K_matrix[5]
            H, W = cv_depth.shape

            points = []
            
            for v in range(H):
                for u in range(W):
                    depth_mm = cv_depth[v, u]
                    if depth_mm == 0: continue
                    
                    Z = float(depth_mm) / DEPTH_SCALE
                    X = (u - cx) * Z / fx
                    Y = (v - cy) * Z / fy
                    
                    b_orig, g_orig, r_orig = cv_rgb[v, u, :].astype(int)
                    rgb_int = self._encode_rgb_to_uint32(r_orig, g_orig, b_orig)

                    points.append((X, Y, Z, rgb_int))
            
            pcd_fields = [
                PointField('x', 0, PointField.FLOAT32, 1), PointField('y', 4, PointField.FLOAT32, 1),
                PointField('z', 8, PointField.FLOAT32, 1), PointField('rgb', 12, PointField.UINT32, 1)
            ]
            
            header = rospy.Header()
            header.frame_id = FRAME_ID 
            return pc2.create_cloud(header, pcd_fields, points)

        except Exception as e:
            rospy.logerr(f"生成点云失败: {e}")
            return None

    def _update_current_frame_data(self):
        """根据 self.current_index 更新正在发布的消息和点云。"""
        if self.max_index == 0: return

        # 确保索引在有效范围内
        index_to_use = self.current_index % self.max_index
        
        # 从列表中取出存储的消息
        rgb_msg, depth_msg, info_msg = self.sync_frames_list[index_to_use]
        
        # 1. 更新存储的 ROS 消息
        self.rgb_msg = rgb_msg
        self.depth_msg = depth_msg
        self.info_msg = info_msg
        
        # 2. 生成点云
        self.pc_raw_msg = self._generate_raw_point_cloud(rgb_msg, depth_msg, info_msg)

        rospy.loginfo(f"--- 帧数据已更新为索引 {index_to_use} ---")

    def _cycle_timer_callback(self, event):
        """每 10 秒触发一次，更新帧索引。"""
        self.current_index += FRAME_INDEX_INCREMENT
        self._update_current_frame_data()

    def _publish_tf(self, current_time):
        t = TransformStamped()
        t.header.stamp = current_time
        t.header.frame_id = MAP_FRAME_ID
        t.child_frame_id = FRAME_ID
        t.transform.translation.x = 0.0
        t.transform.translation.y = 0.0
        t.transform.translation.z = 0.0
        t.transform.rotation.x = 0.0
        t.transform.rotation.y = 0.0
        t.transform.rotation.z = 0.0
        t.transform.rotation.w = 1.0 
        self.tf_broadcaster.sendTransform(t)

    def run_publish_loop(self):
        """高频发布循环，发布当前帧数据。"""
        if self.rgb_msg is None or self.pc_raw_msg is None:
            rospy.logerr("缺少数据，无法启动发布循环。")
            return

        rate = rospy.Rate(PUBLISH_RATE_HZ)
        rospy.loginfo(f"开始以 {PUBLISH_RATE_HZ} Hz 持续发布当前帧数据。")

        while not rospy.is_shutdown():
            current_time = rospy.Time.now()
            
            # 1. 更新所有 Header 的时间戳
            self.rgb_msg.header.stamp = current_time
            self.depth_msg.header.stamp = current_time
            self.info_msg.header.stamp = current_time
            self.pc_raw_msg.header.stamp = current_time
            
            # 2. 发布 TF 变换
            self._publish_tf(current_time)
            
            # 3. 发布传感器图像和内参
            self.pub_rgb.publish(self.rgb_msg)
            self.pub_depth.publish(self.depth_msg)
            self.pub_info.publish(self.info_msg)
            
            # 4. 发布原始点云
            self.pub_pcd_raw.publish(self.pc_raw_msg)
            
            rate.sleep()

if __name__ == '__main__':
    try:
        publisher = BagFramePublisher()
        publisher.run_publish_loop()
    except rospy.ROSInterruptException:
        rospy.loginfo("ROS 中断，Bag 发布节点退出。")

[INFO] [1764753511.075371]: 开始离线提取所有同步帧...
[INFO] [1764753515.570224]: 成功加载 1385 帧同步数据。
[INFO] [1764753516.604050]: --- 帧数据已更新为索引 50 ---
[INFO] [1764753516.605448]: 数据循环定时器启动，每 10 秒切换下一帧。
[INFO] [1764753516.606017]: 开始以 10 Hz 持续发布当前帧数据。
[INFO] [1764753527.647618]: --- 帧数据已更新为索引 100 ---
[INFO] [1764753537.643769]: --- 帧数据已更新为索引 150 ---
[INFO] [1764753547.662670]: --- 帧数据已更新为索引 200 ---
[INFO] [1764753557.715410]: --- 帧数据已更新为索引 250 ---
[INFO] [1764753567.773009]: --- 帧数据已更新为索引 300 ---
[INFO] [1764753577.669293]: --- 帧数据已更新为索引 350 ---
[INFO] [1764753587.591455]: --- 帧数据已更新为索引 400 ---
[INFO] [1764753597.621669]: --- 帧数据已更新为索引 450 ---
[INFO] [1764753607.608323]: --- 帧数据已更新为索引 500 ---
[INFO] [1764753617.540808]: --- 帧数据已更新为索引 550 ---
[INFO] [1764753627.547261]: --- 帧数据已更新为索引 600 ---
[INFO] [1764753637.652896]: --- 帧数据已更新为索引 650 ---
[INFO] [1764753647.559055]: --- 帧数据已更新为索引 700 ---
[INFO] [1764753657.552449]: --- 帧数据已更新为索引 750 ---
[INFO] [1764753667.498922]: --- 帧数据已更新为索引 800 ---
[INFO] [17647

[INFO] [1764753899.261416]: --- 帧数据已更新为索引 565 ---
